<p><font size="6" color='grey'> <b>
Generative KI
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Multi-Agent System für Recherche und Analyse

---

**Aufgabenbeschreibung:**


Dieses Notebook beschreibt die Entwicklung eines Multi-Agent-Systems für automatisierte Recherche, Analyse und Fact-Checking. Das System nutzt moderne LangChain-Komponenten und kombiniert verschiedene spezialisierte Agenten für einen ganzheitlichen Informationsverarbeitungsprozess.



**Ziele des Projekts**


- Entwicklung eines modularen Multi-Agent-Systems für automatisierte Recherche
- Integration von Websuche, Analyse und Fact-Checking in einem System
- Nutzung moderner LLM-Technologien für präzise Informationsverarbeitung

---



**Systemarchitektur**
- Strikte Trennung der Agent-Zuständigkeiten
- Modularer Aufbau für einfache Erweiterbarkeit
- Zentralisierte Konfiguration
- Robuste Fehlerbehandlung



**Best Practices**
- Ausführliche Dokumentation aller Komponenten
- Typisierung für bessere Code-Qualität
- Einheitliche Formatierung und Coding-Standards
- Systematische Fehlerbehandlung

---



**Kernkomponenten**
- **LangChain**: Framework für LLM-Integration (>= 0.2.0)
- **OpenAI**: GPT-3.5-Turbo als Basis-Modell
- **Google Search**: Websuche-Integration vis SerpAPI ([Link](https://serpapi.com/))



**Zusätzliche Tools**
- **Google Colab**: Entwicklungsumgebung
- **HTTPX**: Moderne HTTP-Client-Bibliothek
- **Typing**: Statische Typ-Annotationen

# **1 <font color='orange'>|</font> Setup und Installation**



## **Dependencies**

In [ ]:
%%writefile requirements.txt
langchain>=0.2.0
langchain-community>=0.0.1
langchain-openai>=0.0.1
openai>=1.0.0
duckduckgo-search>=0.0.1
httpx>=0.24.1
anyio==3.7.1
google-auth==2.27.0
gspread>=5.10.0
google-search-results==2.4.2

In [ ]:
# Installation der Requirements
!uv pip install -q -U --system -r requirements.txt

## **Imports**

In [ ]:
# Standardbibliotheken
import os
import json
from datetime import datetime
from dataclasses import dataclass
from typing import Dict, Any, List

# Drittanbieterbibliotheken
from google.colab import userdata
from serpapi import GoogleSearch

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
# from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import Tool

---

# **2 <font color='orange'>|</font> System-Konfiguration**

In [ ]:
@dataclass
class Config:
    """Zentrale Konfiguration für das Multi-Agent System"""
    model_name: str = "gpt-3.5-turbo"
    temperature: float = 0.7
    api_key: str = userdata.get('OpenAI-API-Key')
    serp_api_key: str = userdata.get('SERP_API')

config = Config()
os.environ["OPENAI_API_KEY"] = config.api_key

---

# **3 <font color='orange'>|</font> Agent-Implementierungen**



## **Research Agent**

In [ ]:
# Lösung via LangChain hat zu timeout geführt. Erneuter Versuch in 01.2025
class SerpAPIWrapper:
    """Wrapper um die SerpAPI zu verwenden."""

    def __init__(self):
        self.serp_api_key = config.serp_api_key

    def run(self, query):
        params = {
            "api_key": self.serp_api_key,
            "engine": "google",
            "q": query,
            "num": 5,
            "location": "Austin, Texas, United States",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en"
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        # Extrahiere relevante Informationen aus den Ergebnissen
        answer = ""
        for result in results["organic_results"]:
            answer += f"{result['title']}\n{result['snippet']}\n{result['link']}\n\n"
        return answer

In [ ]:
class ResearchAgent:
    """Agent für die Informationsrecherche mit SerpAPI und Google Search"""

    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.7)  # Beispielwert für temperature
        self.search_tool = SerpAPIWrapper()

        self.tools = [
            Tool(
                name="google_search",
                func=self.search_tool.run,
                description="🔍 Sucht nach aktuellen Informationen in Google"
            )
        ]

        # Agent erstellen
        self.agent = create_openai_tools_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=ChatPromptTemplate.from_messages([
                SystemMessagePromptTemplate.from_template(
                    "Du bist ein Recherche-Spezialist. "
                    "Deine Aufgabe ist es, fundierte Informationen zu sammeln "
                    "und dabei die exakten Quellen anzugeben. "
                    "Wichtig: Gib IMMER die vollständigen URLs der Quellen an, "
                    "keine Platzhalter wie (URL1) oder ähnliches. "
                    "Format: [Information] (Quelle: https://...)"
                ),
                HumanMessagePromptTemplate.from_template("{input}"),
                SystemMessagePromptTemplate.from_template(
                    "Denke Schritt für Schritt vor und dokumentiere die genauen Quellen:\n{agent_scratchpad}"
                )
            ])
        )
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True
        )

    def research(self, topic: str) -> Dict[str, Any]:
        """
        Führt die Recherche durch und erfasst die Quellen

        Args:
            topic: Das Recherchethema

        Returns:
            Dictionary mit Rechercheergebnissen und Quellen
        """
        result = self.agent_executor.invoke({
            "input": f"Recherchiere zum Thema: {topic}. "
            "Gib für jede Information die Quelle (URL) an."
        })

        import re
        urls = re.findall(r'https?://\S+', result['output'])

        return {
            "content": result['output'],
            "sources": urls
        }

In [ ]:
# Test Research Agent
# if __name__ == "__main__":
#     research_agent = ResearchAgent()
#     research_results = research_agent.research("Generative KI Trends 2024")
#     print(research_results["content"])
#     print(research_results["sources"])

## **Analysis Agent**

In [ ]:
class AnalysisAgent:
    """Agent für die Analyse von Rechercheergebnissen"""

    def __init__(self):
        self.llm = ChatOpenAI(temperature=config.temperature)

        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Du bist ein Analyse-Experte. Analysiere die gegebenen "
                "Informationen systematisch und identifiziere Kernaussagen."
            ),
            HumanMessagePromptTemplate.from_template(
                "Analysiere folgende Rechercheergebnisse:\n{research_results}"
            )
        ])

        self.chain = self.prompt | self.llm | StrOutputParser()

    def analyze(self, research_results: str) -> str:
        """📊 Führt die Analyse durch"""
        return self.chain.invoke({"research_results": research_results})

## **FactCheck Agent**

In [ ]:
class FactCheckAgent:
    """Agent für die Überprüfung von Fakten"""

    def __init__(self):
        self.llm = ChatOpenAI(temperature=0.2)

        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Du bist ein Fact-Checking-Experte. Überprüfe die Informationen "
                "kritisch auf ihre Richtigkeit und Konsistenz."
            ),
            HumanMessagePromptTemplate.from_template(
                "Überprüfe folgende Informationen:\n"
                "Recherche: {research_results}\n"
                "Analyse: {analysis}"
            )
        ])

        self.chain = self.prompt | self.llm | StrOutputParser()

    def verify(self, research_results: str, analysis: str) -> str:
        """✨ Führt das Fact-Checking durch"""
        return self.chain.invoke({
            "research_results": research_results,
            "analysis": analysis
        })

---

# **4 <font color='orange'>|</font> Multi-Agent-Koordination**

In [ ]:
class MultiAgentSystem:
    """Koordiniert die Zusammenarbeit der drei Agenten"""

    def __init__(self):
        self.research_agent = ResearchAgent()
        self.analysis_agent = AnalysisAgent()
        self.fact_check_agent = FactCheckAgent()

    def process_topic(self, topic: str) -> Dict[str, Any]:
        """
        Führt den kompletten Verarbeitungsprozess mit allen Agenten durch

        Args:
            topic: Das zu recherchierende Thema

        Returns:
            Dictionary mit den Ergebnissen aller Agenten
        """
        research_results = self.research_agent.research(topic)
        if not research_results['content']:
            raise ValueError("Keine Rechercheergebnisse erhalten")

        analysis_results = self.analysis_agent.analyze(research_results['content'])
        if not analysis_results:
            raise ValueError("Keine Analyseergebnisse erhalten")

        fact_check_results = self.fact_check_agent.verify(
            research_results['content'],
            analysis_results
        )
        if not fact_check_results:
            raise ValueError("Keine Fact-Check-Ergebnisse erhalten")

        return {
            "topic": topic,
            "timestamp": datetime.now().isoformat(),
            "research_results": research_results['content'],
            "sources": research_results['sources'],
            "analysis": analysis_results,
            "fact_check": fact_check_results
        }

---

# **5 <font color='orange'>|</font> Ausgabe und Reporting**

In [ ]:
def generate_report(results: Dict[str, Any]) -> str:
    """
    Erstellt einen formatierten Bericht mit Quellenangaben

    Args:
        results: Dictionary mit allen Ergebnissen und Quellen

    Returns:
        Formatierter Bericht als String
    """
    sources_text = "\n".join([f"- {src}" for src in results['sources']]) if results['sources'] else "Keine externen Quellen gefunden"

    return f"""
=== Recherche- und Analysebericht ===

Thema: {results['topic']}
Erstellt am: {results['timestamp']}

=== 1. 🔍 Rechercheergebnisse ===
{results['research_results']}

=== 2. 📊 Analyse ===
{results['analysis']}

=== 3. ✨ Fact-Checking ===
{results['fact_check']}

=== 4. ♨️ Quellenverzeichnis ===
{sources_text}
"""

---

# **6 <font color='orange'>|</font> Systemtest und Demonstration**

In [ ]:
def main():
    """Hauptfunktion zur Demonstration des Systems"""
    system = MultiAgentSystem()
    topic = "Was sind die aktuelle Trends 2025 im Bereich der generativen KI?"

    try:
        results = system.process_topic(topic)
        report = generate_report(results)
        print(report)
    except Exception as e:
        print(f"Fehler bei der Verarbeitung: {str(e)}")

if __name__ == "__main__":
    main()